<a href="https://colab.research.google.com/github/srijita1406/Machine-Learning-Projects/blob/main/Lug_boot_safety_evaluation_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [30]:
df=pd.read_csv('v4.csv')

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Buying      1496 non-null   object
 1   Maint       1500 non-null   object
 2   Doors       1500 non-null   int64 
 3   Persons     1500 non-null   int64 
 4   Lug_boot    1500 non-null   object
 5   Safety      1500 non-null   object
 6   Evaluation  1500 non-null   object
dtypes: int64(2), object(5)
memory usage: 82.2+ KB


In [32]:
df['Lug_boot'].unique()

array(['med', 'small', 'big'], dtype=object)

In [33]:
y=df['Evaluation']
X=df.drop(['Evaluation'],axis=1)

In [34]:
y.value_counts()

unacc    1039
acc       344
good       59
vgood      58
Name: Evaluation, dtype: int64

In [35]:
from sklearn.impute import SimpleImputer
si=SimpleImputer(strategy='most_frequent')
X=si.fit_transform(X)

In [36]:
type(X)

numpy.ndarray

In [37]:
X=pd.DataFrame(X,columns=['Buying','Maint','Doors','Persons','Lug_boot','Safety'])

In [38]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Buying    1500 non-null   object
 1   Maint     1500 non-null   object
 2   Doors     1500 non-null   object
 3   Persons   1500 non-null   object
 4   Lug_boot  1500 non-null   object
 5   Safety    1500 non-null   object
dtypes: object(6)
memory usage: 70.4+ KB


In [39]:
from sklearn.preprocessing import OneHotEncoder
X1=X.loc[:,['Buying','Maint','Lug_boot','Safety']]
ohe=OneHotEncoder(sparse=False)
X1=ohe.fit_transform(X1)


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [40]:
type(X1)

numpy.ndarray

In [41]:
from sklearn.preprocessing import StandardScaler
X2=X.loc[:,['Doors','Persons']]
ss=StandardScaler()
X2=ss.fit_transform(X2)


In [42]:
type(X2)

numpy.ndarray

In [43]:
X=np.concatenate((X1,X2),axis=1)

In [44]:
X.shape

(1500, 16)

In [45]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=32)

In [46]:
X_train.shape

(1200, 16)

In [47]:
from sklearn.linear_model import Perceptron

In [48]:
pct=Perceptron(random_state=32,eta0=1,max_iter=10,shuffle=False,fit_intercept=True,validation_fraction=0.2,penalty=None,warm_start=True)

In [49]:
pct.fit(X_train,y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Perceptron(eta0=1, max_iter=10, random_state=32, shuffle=False,
           validation_fraction=0.2, warm_start=True)

In [50]:
pct.coef_[0][1]

-1.0

In [54]:
pct2=Perceptron(random_state=32,eta0=2,max_iter=5,shuffle=False,fit_intercept=True,validation_fraction=0.1,penalty=None,warm_start=False)

In [55]:
pct2.fit(X_train,y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Perceptron(eta0=2, max_iter=5, random_state=32, shuffle=False)

In [56]:
pct2.intercept_

array([ -4., -12.,   8., -20.])

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
lr=LogisticRegression(solver='saga',random_state=32,tol=0.01,max_iter=200)
param_dist={'penalty':['l1','l2'],'C':[0.5,1.0,1.5,2.0,2.5,3,3.5,4]}
rcv=RandomizedSearchCV(estimator=lr,param_distributions=param_dist)
rcv.fit(X_train,y_train)
rcv.best_params_

{'penalty': 'l2', 'C': 4}

In [66]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svc=SVC()
param_grid={'kernel':('linear','rbf'),'C':[1,10]}
gscv=GridSearchCV(estimator=svc,param_grid=param_grid)
gscv.fit(X_train,y_train)
gscv.best_params_

{'C': 10, 'kernel': 'rbf'}

In [73]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(metric='minkowski',p=2)
param_grids={'n_neighbors':[1,3,5,7]}
gcv=GridSearchCV(estimator=knn,param_grid=param_grids,cv=4,scoring='accuracy')
gcv.fit(X_train,y_train)


GridSearchCV(cv=4, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7]}, scoring='accuracy')

In [74]:
gcv.best_params_

{'n_neighbors': 7}

In [75]:
gcv.best_score_

0.9083333333333332

In [79]:
from sklearn.metrics import accuracy_score

In [80]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(criterion='gini',splitter='random',min_samples_split=5,min_impurity_decrease=0.001,random_state=32)
dtc.fit(X_train,y_train)
y_pred=dtc.predict(X_test)
accuracy_score(y_test,y_pred)


0.95

In [85]:
dtc.tree_.max_depth

11

In [86]:
dtc.tree_.node_count

81

In [89]:
right_child=tree.children_right[0]

In [90]:
dtc.tree_.impurity[right_child]

0.0

In [94]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# define a dictionary of hyperparameters
param_gridss={'n_estimators':[10,11,12,13,14,15]}

# create a random forest classifier object
rfc = RandomForestClassifier(random_state=32)

# create a GridSearchCV object
grid_search = GridSearchCV(estimator=rfc, param_grid=param_gridss)
grid_search.fit(X_train,y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=32),
             param_grid={'n_estimators': [10, 11, 12, 13, 14, 15]})

In [95]:
grid_search.best_params_

{'n_estimators': 14}

In [96]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
#estimators={'estimators':[SVC,LogisticRegression,KNeighborsClassifier,DecisionTreeClassifier]}
svc=SVC(random_state=32)
bgc=BaggingClassifier(estimator=svc,n_estimators=15,random_state=32)
bgc.fit(X_train,y_train)
y_pred=bgc.predict(X_test)
accuracy_score(y_test,y_pred)

0.9533333333333334

In [98]:
from sklearn.utils.optimize import line_search_wolfe1
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
#estimators={'estimators':[SVC,LogisticRegression,KNeighborsClassifier,DecisionTreeClassifier]}
lr1=LogisticRegression(random_state=32)
bgc=BaggingClassifier(estimator=lr1,n_estimators=15,random_state=32)
bgc.fit(X_train,y_train)
y_pred=bgc.predict(X_test)
accuracy_score(y_test,y_pred)

0.81

In [100]:
from sklearn.utils.optimize import line_search_wolfe1
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
#estimators={'estimators':[SVC,LogisticRegression,KNeighborsClassifier,DecisionTreeClassifier]}
knn1=KNeighborsClassifier()
bgc=BaggingClassifier(estimator=knn1,n_estimators=15,random_state=32)
bgc.fit(X_train,y_train)
y_pred=bgc.predict(X_test)
accuracy_score(y_test,y_pred)

0.93

In [101]:
from sklearn.utils.optimize import line_search_wolfe1
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
#estimators={'estimators':[SVC,LogisticRegression,KNeighborsClassifier,DecisionTreeClassifier]}
dtc1=DecisionTreeClassifier(random_state=32)
bgc=BaggingClassifier(estimator=dtc1,n_estimators=15,random_state=32)
bgc.fit(X_train,y_train)
y_pred=bgc.predict(X_test)
accuracy_score(y_test,y_pred)

0.97

The Decision Tree Classifier gave 97% accuracy with bagging